<a href="https://colab.research.google.com/github/simbakm/training_yolov11_notebook/blob/main/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **starting Point @ simbakm Training YOlo  version 11 bold text**

# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://clickpy.clickhouse.com/dashboard/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
!uv pip install ultralytics
!pip install roboflow kagglehub
import ultralytics
ultralytics.checks()


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/112.6 GB disk)


**STEP 2 — Download the Roboflow dataset (YOLOv11-ready)bold text**

**helper fuction to couint the number of files**

In [2]:
import os
from collections import defaultdict

def count_file_types(dataset_path):
    file_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # get file extension
            file_counts[ext] += 1

    print("File type counts:\n")
    for ext, count in sorted(file_counts.items()):
        print(f"{ext if ext else 'No Extension'} : {count}")

    return dict(file_counts)


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="YdrZFjwWblcZBkXrqqaR")
project = rf.workspace("stairs-ttqqp").project("trash-bin-asn0s")
dataset = project.version(1).download("yolov11")

roboflow_path = dataset.location
print("Roboflow dataset:", roboflow_path)

counts = count_file_types(roboflow_path)



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to trash-bin-1 in yolov11:: 100%|██████████| 902/902 [00:00<00:00, 8563.64it/s]


Roboflow dataset: /content/trash-bin-1
File type counts:

.jpg : 445
.txt : 447
.yaml : 1


## **STEP 3: Download the TACO dataset from KaggleHub**

In [4]:
import kagglehub
path = kagglehub.dataset_download("kneroma/tacotrashdataset")
print("Downloaded to:", path)
counts = count_file_types(path)


100%|██████████| 2.79G/2.79G [00:30<00:00, 97.5MB/s]

Extracting files...


Downloaded to: /root/.cache/kagglehub/datasets/kneroma/tacotrashdataset/versions/3
File type counts:

.bin : 1
.csv : 1
.jpg : 1500
.json : 1
.txt : 1


**STEP 4 — Convert TACO (COCO JSON) → YOLO format**

In [5]:
import json, os, shutil
from glob import glob
from tqdm import tqdm

# TACO dataset path (from KaggleHub)
taco_path = path
data_dir = os.path.join(taco_path, "data")

# Load annotations
annotations_path = os.path.join(data_dir, "annotations.json")
annotations = json.load(open(annotations_path))

# Output
output_taco = "/content/taco_yolo"
#delete privous derectories
if os.path.exists(output_taco):
    shutil.rmtree(output_taco)
    print("Old taco_yolo directory removed.")
else:
    print("No previous taco_yolo folder found.")

os.makedirs(f"{output_taco}/images", exist_ok=True)
os.makedirs(f"{output_taco}/labels", exist_ok=True)

# Map image id → image info
image_map = {img["id"]: img for img in annotations["images"]}

# Find all images recursively
all_images = {}
image_files = glob(f"{data_dir}/**/*.jpg", recursive=True) + \
              glob(f"{data_dir}/**/*.JPG", recursive=True)

for f in image_files:
    all_images[os.path.relpath(f, data_dir)] = f


print("Total images found:", len(all_images))

# Group annotations by image ID
ann_by_image = {}
for ann in annotations["annotations"]:
    img_id = ann["image_id"]
    ann_by_image.setdefault(img_id, []).append(ann)

# Convert to YOLO
for img_id, img_info in tqdm(image_map.items()):
    filename = img_info["file_name"]
    batch_relative_path = filename  # contains batch_x/000000.jpg

    if batch_relative_path not in all_images:
        print("Missing image:", batch_relative_path)
        continue

    src = all_images[batch_relative_path]

    # create unique new filename based on img_id
    new_name = f"{img_id}.jpg"
    dst = f"{output_taco}/images/{new_name}"

    shutil.copy(src, dst)

    # image width/height
    W = img_info["width"]
    H = img_info["height"]

    # label path (same name as image)
    label_path = f"{output_taco}/labels/{img_id}.txt"
    with open(label_path, "w") as f:
        for ann in ann_by_image.get(img_id, []):
            x, y, w, h = ann["bbox"]

            # convert to YOLO
            xc = (x + w/2) / W
            yc = (y + h/2) / H
            w /= W
            h /= H
            # Create category id mapping
            categories = annotations["categories"]
            cat_id_map = {cat["id"]: i for i, cat in enumerate(categories)}
            class_id = cat_id_map[ann["category_id"]]

            f.write(f"{class_id} {xc} {yc} {w} {h}\n")

print("DONE — TACO converted to YOLO:", output_taco)


No previous taco_yolo folder found.
Total images found: 1500


100%|██████████| 1500/1500 [00:13<00:00, 113.18it/s]

DONE — TACO converted to YOLO: /content/taco_yolo


# **5: Split the dataset into Train,val and test using ratio 80|10|10**

In [6]:
import os, random, shutil
from glob import glob

taco_img = "/content/taco_yolo/images"
taco_lbl = "/content/taco_yolo/labels"

images = sorted(glob(f"{taco_img}/*.jpg"))
random.shuffle(images)

n = len(images)
train_split = int(n * 0.8)
val_split = int(n * 0.9)

train_files = images[:train_split]
val_files   = images[train_split:val_split]
test_files  = images[val_split:]

def copy_split(files, folder):
    os.makedirs(f"/content/taco_split/images/{folder}", exist_ok=True)
    os.makedirs(f"/content/taco_split/labels/{folder}", exist_ok=True)
    for img in files:
        lbl = img.replace("images", "labels").replace(".jpg", ".txt")
        shutil.copy(img, f"/content/taco_split/images/{folder}/")
        shutil.copy(lbl, f"/content/taco_split/labels/{folder}/")

copy_split(train_files, "train")
copy_split(val_files, "val")
copy_split(test_files, "test")

print("TACO split complete!")


TACO split complete!


**STEP 6 — Create a combined dataset**

In [7]:
import glob

rf_labels = glob.glob(f"{roboflow_path}/**/labels/*.txt", recursive=True)

for file in rf_labels:
    lines = []
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0:
                parts[0] = "60"  # move to class 60
                lines.append(" ".join(parts))

    with open(file, "w") as f:
        f.write("\n".join(lines))

print("Roboflow labels shifted to class 60")


Roboflow labels shifted to class 60


In [8]:
# remove old combined folder
shutil.rmtree("/content/combined", ignore_errors=True)

# create combined structure
for split in ["train", "val", "test"]:
    os.makedirs(f"/content/combined/images/{split}", exist_ok=True)
    os.makedirs(f"/content/combined/labels/{split}", exist_ok=True)

# copy TACO train/val/test
!cp -r /content/taco_split/images/train/* /content/combined/images/train/
!cp -r /content/taco_split/labels/train/* /content/combined/labels/train/

!cp -r /content/taco_split/images/val/* /content/combined/images/val/
!cp -r /content/taco_split/labels/val/* /content/combined/labels/val/

!cp -r /content/taco_split/images/test/* /content/combined/images/test/
!cp -r /content/taco_split/labels/test/* /content/combined/labels/test/

# copy roboflow
# train
!cp -r $roboflow_path/train/images/* /content/combined/images/train/
!cp -r $roboflow_path/train/labels/* /content/combined/labels/train/

# val
!cp -r $roboflow_path/valid/images/* /content/combined/images/val/
!cp -r $roboflow_path/valid/labels/* /content/combined/labels/val/

# test
!cp -r $roboflow_path/test/images/* /content/combined/images/test/
!cp -r $roboflow_path/test/labels/* /content/combined/labels/test/



**step7: Create YOLO data.yaml**

In [9]:
data_yaml = """
train: /content/combined/images/train
val: /content/combined/images/val
test: /content/combined/images/test

nc: 61
names:
"""


In [11]:
# Load TACO categories
import json
annotations_path = os.path.join(path, "data", "annotations.json")
with open(annotations_path, "r") as f:
    annotations = json.load(f)

category_names = [cat["name"] for cat in annotations["categories"]]

# Add trash_bin as last class
category_names.append("trash_bin")

# Write YAML
with open("combined.yaml", "w") as f:
    f.write("train: /content/combined/images/train\n")
    f.write("val: /content/combined/images/val\n")
    f.write("test: /content/combined/images/test\n\n")
    f.write("nc: 61\n")
    f.write("names:\n")
    for name in category_names:
        f.write(f"  - {name}\n")

print("combined.yaml created correctly with 61 classes.")


combined.yaml created correctly with 61 classes.


# **8: — Train YOLOv11**

In [12]:
!yolo train model=yolo11n.pt data=combined.yaml epochs=50 imgsz=640


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

## **STEP 6 — Download trained model**

In [13]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 1. Predict

YOLO11 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLO11 Predict Docs](https://docs.ultralytics.com/modes/train/).


In [ ]:
# Run inference on an image with YOLO11n
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/zidane.jpg'

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 68.9ms
Speed: 13.1ms preprocess, 68.9ms inference, 250.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/212889447-69e5bdf1-5800-4e29-835e-2ed2336dede2.jpg" width="600">

# 2. Val
Validate a model's accuracy on the [COCO](https://docs.ultralytics.com/datasets/detect/coco/) dataset's `val` or `test` splits. The latest YOLO11 [models](https://github.com/ultralytics/ultralytics#models) are downloaded automatically the first time they are used. See [YOLO11 Val Docs](https://docs.ultralytics.com/modes/val/) for more information.

In [ ]:
# Download COCO val
from ultralytics.utils.downloads import download

download('https://ultralytics.com/assets/coco2017val.zip', unzip=True, dir='datasets') # download (780MB - 5000 images)

In [ ]:
# Validate YOLO11n on COCO8 val
!yolo val model=yolo11n.pt data=coco8.yaml

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

WARNING ⚠️ Dataset 'coco8.yaml' images not found, missing path '/content/datasets/coco8/images/val'
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100% ━━━━━━━━━━━━ 25/25 4962.5files/s 0.0s
Dataset download success ✅ (0.6s), saved to /content/datasets

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1486.2±504.6 MB/s, size: 54.0 KB)
val: Scanning /content/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4/4 118.2it/s 0.0s
val: New cache created: /content/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.88it/s 1.1s
                   all          4         17       0.57       0.85      0.847      0.632
                person          3         10      0.557        0.6   

# 3. Train

<p align=""><a href="https://ultralytics.com/hub"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/yolov8/banner-integrations.png"/></a></p>

Train YOLO11 on [Detect](https://docs.ultralytics.com/tasks/detect/), [Segment](https://docs.ultralytics.com/tasks/segment/), [Classify](https://docs.ultralytics.com/tasks/classify/) and [Pose](https://docs.ultralytics.com/tasks/pose/) datasets. See [YOLO11 Train Docs](https://docs.ultralytics.com/modes/train/) for more information.

In [ ]:
#@title Select YOLO11 🚀 logger {run: 'auto'}
logger = 'TensorBoard' #@param ['TensorBoard', 'Weights & Biases']

if logger == 'TensorBoard':
  !yolo settings tensorboard=True
  %load_ext tensorboard
  %tensorboard --logdir .
elif logger == 'Weights & Biases':
  !yolo settings wandb=True

In [ ]:
# Train YOLO11n on COCO8 for 3 epochs
!yolo train model=yolo11n.pt data=coco8.yaml epochs=3 imgsz=640

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

# 4. Export

Export a YOLO model to any supported format below with the `format` argument, i.e. `format=onnx`. See [Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://docs.ultralytics.com/integrations/onnx/) or [OpenVINO](https://docs.ultralytics.com/integrations/openvino/) for up to 3x CPU speedup.
- 💡 ProTip: Export to [TensorRT](https://docs.ultralytics.com/integrations/tensorrt/) for up to 5x GPU speedup.

| Format | `format` Argument | Model | Metadata | Arguments |
|--------|-----------------|-------|----------|------------|
| [PyTorch](https://pytorch.org/) | - | `yolo11n.pt` | ✅ | - |
| [TorchScript](https://docs.ultralytics.com/integrations/torchscript) | `torchscript` | `yolo11n.torchscript` | ✅ | `imgsz`, `batch`, `dynamic`, `optimize`, `half`, `nms`, `device` |
| [ONNX](https://docs.ultralytics.com/integrations/onnx) | `onnx` | `yolo11n.onnx` | ✅ | `imgsz`, `batch`, `dynamic`, `half`, `opset`, `simplify`, `nms`, `device` |
| [OpenVINO](https://docs.ultralytics.com/integrations/openvino) | `openvino` | `yolo11n_openvino_model/` | ✅ | `imgsz`, `batch`, `dynamic`, `half`, `int8`, `nms`, `fraction`, `device`, `data` |
| [TensorRT](https://docs.ultralytics.com/integrations/tensorrt) | `engine` | `yolo11n.engine` | ✅ | `imgsz`, `batch`, `dynamic`, `half`, `int8`, `simplify`, `nms`, `fraction`, `device`, `data`, `workspace` |
| [CoreML](https://docs.ultralytics.com/integrations/coreml) | `coreml` | `yolo11n.mlpackage` | ✅ | `imgsz`, `batch`, `half`, `int8`, `nms`, `device` |
| [TF SavedModel](https://docs.ultralytics.com/integrations/tf-savedmodel) | `saved_model` | `yolo11n_saved_model/` | ✅ | `imgsz`, `batch`, `int8`, `keras`, `nms`, `device` |
| [TF GraphDef](https://docs.ultralytics.com/integrations/tf-graphdef) | `pb` | `yolo11n.pb` | ❌ | `imgsz`, `batch`, `device` |
| [TF Lite](https://docs.ultralytics.com/integrations/tflite) | `tflite` | `yolo11n.tflite` | ✅ | `imgsz`, `batch`, `half`, `int8`, `nms`, `fraction`, `device`, `data` |
| [TF Edge TPU](https://docs.ultralytics.com/integrations/edge-tpu) | `edgetpu` | `yolo11n_edgetpu.tflite` | ✅ | `imgsz`, `device` |
| [TF.js](https://docs.ultralytics.com/integrations/tfjs) | `tfjs` | `yolo11n_web_model/` | ✅ | `imgsz`, `batch`, `half`, `int8`, `nms`, `device` |
| [PaddlePaddle](https://docs.ultralytics.com/integrations/paddlepaddle) | `paddle` | `yolo11n_paddle_model/` | ✅ | `imgsz`, `batch`, `device` |
| [MNN](https://docs.ultralytics.com/integrations/mnn) | `mnn` | `yolo11n.mnn` | ✅ | `imgsz`, `batch`, `half`, `int8`, `device` |
| [NCNN](https://docs.ultralytics.com/integrations/ncnn) | `ncnn` | `yolo11n_ncnn_model/` | ✅ | `imgsz`, `batch`, `half`, `device` |
| [IMX500](https://docs.ultralytics.com/integrations/sony-imx500) | `imx` | `yolo11n_imx_model/` | ✅ | `imgsz`, `int8`, `fraction`, `device`, `data` |
| [RKNN](https://docs.ultralytics.com/integrations/rockchip-rknn) | `rknn` | `yolo11n_rknn_model/` | ✅ | `imgsz`, `batch`, `name`, `device` || [ExecuTorch](https://docs.ultralytics.com/integrations/executorch) | `executorch` | `executorch_model/` | ✅ | `imgsz`, `device` |

In [ ]:
!yolo export model=yolo11n.pt format=torchscript

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

TorchScript: starting export with torch 2.8.0+cu126...
TorchScript: export success ✅ 2.1s, saved as 'yolo11n.torchscript' (10.5 MB)

Export complete (2.6s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo11n.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolo11n.torchscript imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


# 5. Python Usage

YOLO11 was reimagined using Python-first principles for the most seamless Python YOLO experience yet. YOLO11 models can be loaded from a trained checkpoint or created from scratch. Then methods are used to train, val, predict, and export the model. See detailed Python usage examples in the [YOLO11 Python Docs](https://docs.ultralytics.com/usage/python/).

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11n.yaml')  # build a new model from scratch
model = YOLO('yolo11n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco8.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
results = model.export(format='onnx')  # export the model to ONNX format

# 6. Tasks

YOLO11 can train, val, predict and export models for the most common tasks in vision AI: [Detect](https://docs.ultralytics.com/tasks/detect/), [Segment](https://docs.ultralytics.com/tasks/segment/), [Classify](https://docs.ultralytics.com/tasks/classify/) and [Pose](https://docs.ultralytics.com/tasks/pose/). See [YOLO11 Tasks Docs](https://docs.ultralytics.com/tasks/) for more information.

<br><img width="1024" src="https://raw.githubusercontent.com/ultralytics/assets/main/im/banner-tasks.png">


## 1. Detection

YOLO11 _detection_ models have no suffix and are the default YOLO11 models, i.e. `yolo11n.pt` and are pretrained on COCO. See [Detection Docs](https://docs.ultralytics.com/tasks/detect/) for full details.


In [ ]:
# Load YOLO11n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # load a pretrained YOLO detection model
model.train(data='coco8.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 2. Segmentation

YOLO11 _segmentation_ models use the `-seg` suffix, i.e. `yolo11n-seg.pt` and are pretrained on COCO. See [Segmentation Docs](https://docs.ultralytics.com/tasks/segment/) for full details.


In [ ]:
# Load YOLO11n-seg, train it on COCO128-seg for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n-seg.pt')  # load a pretrained YOLO segmentation model
model.train(data='coco8-seg.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 3. Classification

YOLO11 _classification_ models use the `-cls` suffix, i.e. `yolo11n-cls.pt` and are pretrained on ImageNet. See [Classification Docs](https://docs.ultralytics.com/tasks/classify/) for full details.


In [ ]:
# Load YOLO11n-cls, train it on mnist160 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n-cls.pt')  # load a pretrained YOLO classification model
model.train(data='mnist160', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 4. Pose

YOLO11 _pose_ models use the `-pose` suffix, i.e. `yolo11n-pose.pt` and are pretrained on COCO Keypoints. See [Pose Docs](https://docs.ultralytics.com/tasks/pose/) for full details.

In [ ]:
# Load YOLO11n-pose, train it on COCO8-pose for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n-pose.pt')  # load a pretrained YOLO pose model
model.train(data='coco8-pose.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 4. Oriented Bounding Boxes (OBB)

YOLO11 _OBB_ models use the `-obb` suffix, i.e. `yolo11n-obb.pt` and are pretrained on the DOTA dataset. See [OBB Docs](https://docs.ultralytics.com/tasks/obb/) for full details.

In [ ]:
# Load YOLO11n-obb, train it on DOTA8 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n-obb.pt')  # load a pretrained YOLO OBB model
model.train(data='dota8.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/boats.jpg')  # predict on an image

# Appendix

Additional content below.

In [ ]:
# Pip install from source
!uv pip install git+https://github.com/ultralytics/ultralytics@main

In [ ]:
# Git clone and run tests on 'main' branch
!git clone https://github.com/ultralytics/ultralytics -b main
!uv pip install -qe ultralytics

In [ ]:
# Run tests (Git clone only)
!pytest ultralytics/tests

In [ ]:
# Validate multiple models
for x in 'nsmlx':
  !yolo val model=yolo11{x}.pt data=coco.yaml

# Task
The "Missing image" messages are not indicative of an error in the provided code. They appear because some image files referenced in the TACO dataset's `annotations.json` are not present in the downloaded `tacotrashdataset/data` directory. The script correctly handles this by skipping those annotations for which the corresponding image file is missing.

If you intend to use all annotations, you might need to:
1.  **Verify the integrity of the downloaded TACO dataset**: Ensure that the Kaggle dataset "kneroma/tacotrashdataset" you downloaded is complete and all referenced image files are present.
2.  **Manually download missing images**: If there are specific images crucial for your task, you might need to locate and download them separately if they exist elsewhere.

## explain_missing_images

### Subtask:
Explain that the 'Missing image' messages are due to images being absent from the downloaded TACO dataset, not a code error. The script correctly handles this by skipping them.


The 'Missing image' messages you observed in the previous output are not an error in the code's logic. They indicate that some image files referenced in the TACO dataset's 'annotations.json' are not actually present in the '/kaggle/input/tacotrashdataset/data' directory after downloading the dataset from KaggleHub. The script was designed to handle this by skipping the annotations associated with these missing images, allowing the conversion process to continue without crashing.

## propose_dataset_integrity_check

### Subtask:
Suggest that the user might need to verify the integrity of the downloaded TACO dataset or ensure all necessary image files are present if they expect all annotations to have corresponding images.


## Summary:

### Q&A
The 'Missing image' messages observed in the previous output are not indicative of an error in the provided code. Instead, they signify that certain image files referenced in the TACO dataset's `annotations.json` file are not present in the `/kaggle/input/tacotrashdataset/data` directory. The script is designed to handle this situation by skipping the annotations associated with these missing images, thus preventing crashes and allowing the conversion process to continue.

### Data Analysis Key Findings
*   The 'Missing image' messages are a result of discrepancies between the `annotations.json` file and the actual image files available in the downloaded dataset.
*   The script correctly identifies and skips annotations for which corresponding image files are absent, demonstrating robust error handling.

### Insights or Next Steps
*   If the goal is to utilize all annotations, it is crucial to verify the integrity and completeness of the downloaded TACO dataset from Kaggle to ensure all referenced image files are present.
*   Alternatively, if specific missing images are essential, they may need to be located and downloaded manually from external sources.
